In [5]:
import os
import cv2
import numpy as np
from PIL import Image

def generate_artifacts_and_mask(img_path, output_img_path, mask_path, artifact_ratio=0.02, min_size=5, max_size=15):
    """
    为图像添加伪影和噪点,并生成对应的mask
    
    参数:
        img_path: 输入原始图像路径
        output_img_path: 保存添加伪影后的图像路径
        mask_path: 保存mask的路径
        artifact_ratio: 伪影区域占比,取值0-1之间
        min_size: 伪影区域最小尺寸
        max_size: 伪影区域最大尺寸
    """
    # 读取图像
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    
    # 创建空白mask和复制原图用于添加伪影
    mask = np.zeros((height, width), dtype=np.uint8)
    corrupted_img = img.copy()
    
    # 计算需要的伪影区域数量
    total_pixels = height * width
    target_pixels = int(total_pixels * artifact_ratio)
    current_pixels = 0
    
    # 随机生成不规则形状的伪影区域
    while current_pixels < target_pixels:
        # 随机选择区域大小
        size = np.random.randint(min_size, max_size)
        
        # 随机选择位置
        x = np.random.randint(0, width - size)
        y = np.random.randint(0, height - size)
        
        # 生成不规则形状并添加伪影
        for i in range(size):
            for j in range(size):
                if np.random.random() > 0.3:  # 70%概率填充像素
                    if y+i < height and x+j < width:
                        if mask[y+i, x+j] == 0:  # 只处理未处理的像素
                            mask[y+i, x+j] = 255
                            # 添加随机颜色的伪影
                            corrupted_img[y+i, x+j] = np.random.randint(0, 255, 3)
                            current_pixels += 1
        
        # 添加一些随机的小噪点
        if np.random.random() > 0.5:
            noise_x = np.random.randint(0, width)
            noise_y = np.random.randint(0, height)
            if mask[noise_y, noise_x] == 0:
                mask[noise_y, noise_x] = 255
                corrupted_img[noise_y, noise_x] = np.random.randint(0, 255, 3)
                current_pixels += 1
    
    # 使用形态学操作使边缘更自然
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # 创建必要的目录
    os.makedirs(os.path.dirname(output_img_path), exist_ok=True)
    os.makedirs(os.path.dirname(mask_path), exist_ok=True)
    
    # 保存带伪影的图像和mask
    cv2.imwrite(output_img_path, corrupted_img)
    cv2.imwrite(mask_path, mask)
    
    return corrupted_img, mask

# 示例用法
img_dir = "." # 原始图像目录
corrupted_dir = "./corrupted" # 带伪影的图像保存目录
mask_dir = "./mask" # mask保存目录

# 确保目录存在
os.makedirs(img_dir, exist_ok=True)
os.makedirs(corrupted_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# 为目录下所有图像生成带伪影的图像和对应mask
for img_name in os.listdir(img_dir):
    if img_name.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(img_dir, img_name)
        corrupted_path = os.path.join(corrupted_dir, f"corrupted_{img_name}")
        mask_name = os.path.splitext(img_name)[0] + '_mask.png'
        mask_path = os.path.join(mask_dir, mask_name)
        
        generate_artifacts_and_mask(img_path, corrupted_path, mask_path)
        print(f"Generated corrupted image and mask for {img_name}")


Generated corrupted image and mask for image.png
